In [2]:
import os
import numpy as np
import pandas as pd

%config IPCompleter.greedy=True

### Initial datasets

In [3]:
import sqlalchemy as sa

In [4]:

data_name = 'hypermarket.db'
route = '../../02_Data/01_Originals/' + data_name


In [5]:
con = sa.create_engine('sqlite:///' + route)

In [6]:
from sqlalchemy import inspect
insp = inspect(con)
tables = insp.get_table_names()
tables

['calendar', 'sales', 'sell_prices']

In [7]:
cal = pd.read_sql('calendar', con)
cal.drop(columns='index',inplace=True)
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2
0,2013-01-01,11249,Tuesday,4,1,2013,d_704,NewYear,National,None,None
1,2013-01-02,11249,Wednesday,5,1,2013,d_705,None,None,None,None
2,2013-01-03,11249,Thursday,6,1,2013,d_706,None,None,None,None
3,2013-01-04,11249,Friday,7,1,2013,d_707,None,None,None,None
4,2013-01-05,11250,Saturday,1,1,2013,d_708,None,None,None,None


In [8]:
sales = pd.read_sql('sales', con)
sales.drop(columns='index',inplace=True)
sales.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d_704,d_705,d_706,d_707,...,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,0,224,241,232,...,5,2,0,0,6,0,6,0,0,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,54,63,44,0,65,90,104,73,76,97
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,20,23,23,25,...,43,40,39,0,29,33,27,13,26,47
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,34,27,40,49,...,31,43,32,0,52,37,32,29,34,27
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,29,45,28,0,46,36,40,31,46,36


In [9]:
prices = pd.read_sql('sell_prices', con)
prices.drop(columns='index',inplace=True)
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


## Analytic board

In [10]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_704,d_705,d_706,d_707,...,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,0,224,241,232,...,5,2,0,0,6,0,6,0,0,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,54,63,44,0,65,90,104,73,76,97
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,20,23,23,25,...,43,40,39,0,29,33,27,13,26,47
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,34,27,40,49,...,31,43,32,0,52,37,32,29,34,27
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,29,45,28,0,46,36,40,31,46,36


In [11]:
#tabular to transactional --> melt

sales = sales.melt(id_vars = sales.columns[0:6], # from 0 to 5 inclusive (6 excluded)
                  var_name = 'd',
                  value_name = 'sales')


In [14]:
sales.drop(columns='id',inplace=True)

In [15]:
sales

,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0
...,...,...,...,...,...,...,...
21895,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6
21896,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5
21897,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11
21898,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16


In [16]:
# merge with calendar

df = sales.merge(right = cal, how = 'left', on = 'd')

In [17]:
df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None


In [18]:
# include prices

prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


In [20]:
df = df.merge(right = prices, how = 'left', on = ['store_id', 'item_id', 'wm_yr_wk'])

In [21]:
df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.98
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.28
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.48
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,NaN


### Reordering

In [36]:
df.head(1)

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25


In [37]:
order = ['date',
         'state_id',
         'store_id',
         'dept_id',
         'cat_id',
         'item_id',
         'wm_yr_wk',
         'd',
         'sales',
         'sell_price',
         'year',
         'month',
         'wday',
         'weekday',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2']

In [40]:
df = df[order].set_index('date')


In [41]:
df.head(1)

,state_id,store_id,dept_id,cat_id,item_id,wm_yr_wk,d,sales,sell_price,year,month,wday,weekday,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,,,,,,,,,,,,,
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_090,11249,d_704,0,1.25,2013,1,4,Tuesday,NewYear,National,None,None


### Extract validation dataset

In [42]:
# since is forecasting, we validate with the last days

validation = df['2015-12-01':'2015-12-31']
work = df[:'2015-11-31']


In [43]:
validation.to_csv('../../02_Data/02_Validation/validation.csv')
work.to_csv('../../02_Data/03_Work/work.csv')